## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- Lemmatisation
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données

Ajouts (2022-07-12)
- LLR (semble fonctionner maintenant)

### **Lire le corpus** 

In [138]:
import shutil, re, pandas, random
from os import listdir, chdir, path
from pathlib import Path

acteur = 'msss'
sous_corpus = False 
tag = ''

# Change the directory
if sous_corpus:
    base_path = '../03-corpus/2-sous-corpus/'
    file_path = path.join(base_path, acteur, tag)

else: 
    base_path = '../03-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur) + '.csv'

In [139]:
from pandas import *
with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path)
        text = data['text'].tolist()

In [140]:
nb_docs = len(text)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 9747 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [141]:
n = round(0.6 * nb_docs)
corpus = random.sample(text, n)

print("On va travailler sur un échantillon correspondant à environ 40 % des documents du corpus, soit {} documents". format(len(corpus)))

corpus = " ".join([(re.sub('\d', '', t.strip('\n').lower().replace('’', '\''))) for t in text])

On va travailler sur un échantillon correspondant à environ 40 % des documents du corpus, soit 5848 documents


**NLTK**\
https://www.nltk.org/ 

In [142]:
import nltk
#nltk.download(['popular'])

In [143]:
punct = '!#$%&()*+,-/:;<=>?@[\]^_{|}~©'

for t in punct:
    corpus = corpus.replace(t, ' ').replace("  ", " ")

### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [144]:
file_path = '../04-filtrage/mwe_stopwords.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

In [145]:
for mwe in mwe_sw:
    corpus = corpus.replace(mwe, '')

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [146]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

tokens = tokenizer_re.tokenize(corpus)
len_corpus = len(tokens)

print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len_corpus))

Avec le RegExpTokenizer, notre corpus contient 15519892 tokens.


In [147]:
tokens[:50]

['durée',
 'moyenne',
 'de',
 'séjour',
 'pour',
 'la',
 'clientèle',
 'de',
 'ans',
 'et',
 'plus',
 'sur',
 'civière',
 'à',
 "l'",
 'urgence',
 'répertoire',
 'des',
 'indicateurs',
 'de',
 'gestion',
 'en',
 'santé',
 'et',
 'services',
 'sociaux',
 'recherche',
 'québec',
 'ca',
 'ministère',
 'réseau',
 'professionnels',
 'publications',
 'portail',
 'santé',
 'mieux',
 'être',
 'répertoire',
 'des',
 'indicateurs',
 'de',
 'gestion',
 'en',
 'santé',
 'et',
 'services',
 'sociaux',
 'répertoires',
 'répertoire',
 'des']

In [35]:
corpus = " ".join(tokens).replace("' ", "'")

In [36]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [37]:
file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

with open(file_path) as f:
    csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [54]:
# tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]
tagged = [(t.split('\t')[0], mapping[t.split('\t')[1]]) for t in tagger.tag_text(corpus)]


#if len(t.split('\t')) >1

In [55]:
tagged

[('institut', 'nc'),
 ('national', 'adj'),
 ('de', 'prep'),
 ('psychiatrie', 'nc'),
 ('légale', 'adj'),
 ('philippe', 'nc'),
 ('pinel', 'adj'),
 ('a', 'v'),
 ('a', 'v'),
 ('a', 'v'),
 ('événements', 'nc'),
 ('nouvelles', 'adj'),
 ('covid', 'adj'),
 ('carrières', 'nc'),
 ('forensia', 'v'),
 ('fondation', 'nc'),
 ('soins', 'nc'),
 ('aux', 'prep'),
 ('patients', 'adj'),
 ('familles', 'nc'),
 ('et', 'csu'),
 ('visiteurs', 'nc'),
 ('services', 'nc'),
 ('aux', 'prep'),
 ('professionnels', 'adj'),
 ('recherche', 'nc'),
 ('volet', 'nc'),
 ('académique', 'adj'),
 ("l'", 'det'),
 ('institut', 'nc'),
 ('menu', 'adj'),
 ('soins', 'nc'),
 ('aux', 'prep'),
 ('patients', 'nc'),
 ('voir', 'v'),
 ('tout', 'pro'),
 ('philosophie', 'nc'),
 ('des', 'prep'),
 ('soins', 'nc'),
 ('expertise', 'v'),
 ('sécurité', 'nc'),
 ('des', 'prep'),
 ('soins', 'nc'),
 ('équipes', 'nc'),
 ('éthique', 'adj'),
 ('intégration', 'nc'),
 ('sociale', 'adj'),
 ('programmes', 'nc'),
 ('et', 'csu'),
 ('traitements', 'nc'),
 ('prog

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [56]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [57]:
lemmatizer = FrenchLefffLemmatizer()

In [60]:
lemmas = []
for term in tagged:
    term_l = []
    if lemmatizer.lemmatize(term[0], term[1]) == []:
        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
        term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    else:
        term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])
    
    lemmas.append(term_l)

In [61]:
lemmas

[('institut', 'nc'),
 ('national', 'adj'),
 ('de', 'prep'),
 ('psychiatrie', 'nc'),
 ('légal', 'adj'),
 ('philippe', 'nc'),
 ('pinel', 'adj'),
 ('avoir', 'v'),
 ('avoir', 'v'),
 ('avoir', 'v'),
 ('événement', 'nc'),
 ('nouveau', 'adj'),
 ('covid', 'adj'),
 ('carrière', 'nc'),
 ('forensia', 'v'),
 ('fondation', 'nc'),
 ('soin', 'nc'),
 ('aux', 'prep'),
 ('patient', 'adj'),
 ('famille', 'nc'),
 ('et', 'csu'),
 ('visiteur', 'nc'),
 ('service', 'nc'),
 ('aux', 'prep'),
 ('professionnel', 'adj'),
 ('recherche', 'nc'),
 ('volet', 'nc'),
 ('académique', 'adj'),
 ('le', 'det'),
 ('institut', 'nc'),
 ('menu', 'adj'),
 ('soin', 'nc'),
 ('aux', 'prep'),
 ('patient', 'nc'),
 ('voir', 'v'),
 ('tout', 'pro'),
 ('philosophie', 'nc'),
 ('des', 'prep'),
 ('soin', 'nc'),
 ('expertiser', 'v'),
 ('sécurité', 'nc'),
 ('des', 'prep'),
 ('soin', 'nc'),
 ('équipe', 'nc'),
 ('éthique', 'adj'),
 ('intégration', 'nc'),
 ('social', 'adj'),
 ('programme', 'nc'),
 ('et', 'csu'),
 ('traitement', 'nc'),
 ('programme'

### **Filtrage** (antidictionnaire)  

In [62]:
# Importer l'antidictionnaire pour filtrer les données

# Stopwords lemmatisés
file_path = '../04-filtrage/stopwords_lemmatized.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [20]:
ngrammes = list(everygrams(tagged, min_len=2, max_len=5))
ngrammes_lemmatized = list(everygrams(lemmas, min_len=2, max_len=5))

In [22]:
print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))

Avant filtrage, on a 3131973 bigrammes.


### **Filtrage (N-grammes)**

On retire les n-grammes qui apparaissent moins de 30 fois dans tout le corpus ou qui débutent ou terminent par :
- un stopword
- un mot de 1 lettre ou moins

Pour le reste du traitement, on arrête de considérer les frontières entre les phrases et entre les documents (nos ngrammes les respectent donc on n'en a plus besoin)

In [23]:
def extract_patterns(ngrammes):
    patterns = []

    for ng in ngrammes:
        phrase = []
        pattern = []
        for t in ng:
            phrase.append(t[0]) # token
            pattern.append(t[1]) # POS tag

        patterns.append([phrase, pattern])
        
    return patterns

phrases = extract_patterns(ngrammes)
phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [24]:
phrases[:20]

[[['inesss', 'médicaments'], ['adj', 'nc']],
 [['médicaments', 'évaluation'], ['nc', 'nc']],
 [['évaluation', 'aux'], ['nc', 'prep']],
 [['aux', 'fins'], ['prep', 'nc']],
 [['fins', "d'"], ['nc', 'prep']],
 [["d'", 'inscription'], ['prep', 'nc']],
 [['inscription', 'emplois'], ['nc', 'nc']],
 [['emplois', 'english'], ['nc', 'nc']],
 [['english', 'inesss'], ['nc', 'adj']],
 [['inesss', 'express'], ['adj', 'adj']],
 [['express', 'algorithmes'], ['adj', 'nc']],
 [['algorithmes', 'en'], ['nc', 'prep']],
 [['en', 'cancérologie'], ['prep', 'nc']],
 [['cancérologie', 'consulter'], ['nc', 'v']],
 [['consulter', 'une'], ['v', 'det']],
 [['une', 'publication'], ['det', 'nc']],
 [['publication', 'évaluation'], ['nc', 'nc']],
 [['évaluation', 'des'], ['nc', 'prep']],
 [['des', 'médicaments'], ['prep', 'nc']],
 [['médicaments', 'aux'], ['nc', 'prep']]]

In [25]:
freq = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])
freq_lemmatized = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases_lemmatized])

In [26]:
freq

FreqDist({"d'évaluation": 48954, 'en attente': 35656, "attente d'": 35653, 'première demande': 29120, 'de la': 28714, 'en cours': 28355, 'demande en': 26802, "de l'": 24787, "cours d'": 22196, 'traitement de': 19893, ...})

In [27]:
def filtrer_phrases(phrases, freq):
        output = []
        for term in phrases:
                exp = " ".join(term[0]).replace("' ", "'")
                f = freq[exp]
                
                # f > x and - Si on veut filtrer par fréquence 
                if  not term[0][0] in stopwords and len(term[0][0]) > 2 \
                and not term[0][-1] in stopwords and len(term[0][-1]) > 2 : 
                        pattern = " ".join(term[1])            
                        output.append([exp, pattern, f])  
                         
        return output

phrases = filtrer_phrases(phrases, freq)
phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [28]:
print("Après filtrage, on a {} occurrences de bigrammes.".format(len(phrases))) 

Après filtrage, on a 981757 occurrences de bigrammes.


In [29]:
def tabCSV(phrases, titre):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(phrases, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    file_path = path.join(base_path, acteur, acteur)
    if sous_corpus:
       file_path = path.join(base_path, acteur, tag, tag)
    

    Path(file_path).mkdir(parents=True, exist_ok=True)

    tab.to_csv(file_path + titre)

    return tab.values.tolist()

In [30]:
phrases = tabCSV(phrases,'_n-grams.csv')
phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

In [31]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(phrases)))

Après filtrage, on a 2013 bigrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [32]:
file_path = '../04-filtrage/mesh_patterns-fr.csv'

with open (file_path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:100] # Pour prendre seulement les 50 structures syntaxiques les plus fréquentes dans les MeSH

In [33]:
terms = [t for t in phrases if t[1] in patterns]
terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [34]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} bigrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 17 % des termes
On avait 2013 bigrammes, on en a maintenant 1678.


In [35]:
import pandas as pd

def extract_terms(liste_terms, titre):
    file_path = '../04-filtrage/output/'
    tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    if sous_corpus:
        file_path = path.join(file_path, acteur, tag, tag)

    else :
        file_path = path.join(file_path, acteur, acteur)

                    
    tab.to_csv(file_path + titre)

extract_terms(terms, '_terms.csv')
extract_terms(terms_lemmatized, '_terms-lemmatized.csv')

### **Filtrage (Collocations statistiquement significatives)** - Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [36]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 1678 bigrammes.


In [148]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [64]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0].replace("' ", "'") for t in ng]) for ng in ngrammes]

In [65]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [66]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [67]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [68]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)


extrant.to_csv(file_path + '_KWIC' +'.csv')

### **Extraction de termes MeSH**

In [69]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [70]:
extr_mesh = tokenizer_mesh.tokenize([t[0] for t in terms])

In [71]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [72]:
file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [73]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [74]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

In [75]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [76]:
file_path = '../04-filtrage/output' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')